In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import math
import numpy as np
from scipy import stats

In [76]:
#read in curbs_processed.json
curbs = gpd.read_file('curbs_processed.json')
curbs

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,avg_image_capture_date,avg_label_date,severity,is_temporary,agree_count,disagree_count,notsure_count,lat,lng,cluster_id,geometry
0,70839935,CurbRamp,14822,98239835,Harbor Island,2017-09-01,2019-04-26 21:46:21.100,3.0,False,1,0,0,47.584560,-122.352478,9472,POINT (-122.35248 47.58456)
1,70839937,CurbRamp,14809,98239825,Harbor Island,2017-09-01,2019-04-22 00:58:33.914,2.0,False,1,0,0,47.572361,-122.352356,9451,POINT (-122.35236 47.57236)
2,70839941,CurbRamp,14809,98239825,Harbor Island,2017-09-01,2019-04-26 11:45:42.880,3.0,False,2,0,0,47.572842,-122.352943,9449,POINT (-122.35294 47.57284)
3,70839942,CurbRamp,14809,98239825,Harbor Island,2016-01-16,2019-04-26 11:45:33.643,3.0,False,2,0,0,47.572910,-122.353020,9450,POINT (-122.35302 47.57291)
4,70839944,CurbRamp,14833,98239870,Harbor Island,2017-09-01,2019-04-21 23:21:35.297,2.0,False,4,0,0,47.579891,-122.355782,9467,POINT (-122.35578 47.57989)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45043,77221366,NoCurbRamp,8062,6478218,Matthews Beach,2019-06-01,2019-11-29 00:16:40.910,5.0,False,1,0,1,47.701054,-122.287994,585,POINT (-122.28799 47.70105)
45044,77221368,NoCurbRamp,4960,6427434,Matthews Beach,2019-06-01,2023-03-27 16:33:28.207,3.0,False,0,0,0,47.698399,-122.285370,624,POINT (-122.28537 47.69840)
45045,77221370,NoCurbRamp,4960,6427434,Matthews Beach,2018-10-01,2019-04-13 22:40:06.920,NaN,False,5,0,0,47.698444,-122.290504,5327,POINT (-122.29050 47.69844)
45046,77221371,NoCurbRamp,16909,158788694,Matthews Beach,2019-06-01,2019-11-30 23:18:22.315,5.0,False,2,0,0,47.700176,-122.290520,602,POINT (-122.29052 47.70018)


In [4]:
#read in sidewalk data
sidewalk = gpd.read_file('seattle-sidewalk/Sidewalks/Sidewalks.shp')
sidewalk = sidewalk.dropna(subset=['geometry'])

In [78]:
#convert both to projection 2285
curbs = curbs.to_crs(epsg=2285)
sidewalk = sidewalk.to_crs(epsg=2285)

In [82]:
curbs_mapped_on_sidewalk = curbs.sjoin_nearest(sidewalk, distance_col='distance', max_distance = 30)
#curbs_mapped_on_sidewalk
#44848

In [83]:
#count the number of curbs per street
curbs_count_per_street= curbs_mapped_on_sidewalk.groupby(['OBJECTID']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
curbs_count_per_street

,OBJECTID,counts
3252,7793,8
21161,32935,8
6422,12810,8
14326,23256,7
5564,11236,7
...,...,...
11505,19645,1
11508,19648,1
11509,19650,1
11513,19655,1


In [104]:
#select the streets with more than 2 curbs, exactly 2 curbs, and exactly 1 curb
streets_w_more_than_2_curbs= curbs_count_per_street[curbs_count_per_street['counts'] > 2]
streets_w_2_curbs = curbs_count_per_street[curbs_count_per_street['counts'] == 2]
streets_w_1_curb = curbs_count_per_street[curbs_count_per_street['counts'] == 1]

In [97]:
#for streets with exactly 2 curbs, select the OBJECTIDs in the curbs_mapped_on_sidewalk dataframe
curbs_2 = curbs_mapped_on_sidewalk[curbs_mapped_on_sidewalk['OBJECTID'].isin(streets_w_2_curbs['OBJECTID'])]
# curbs_2
# 17172

In [99]:
curbs_1 = curbs_mapped_on_sidewalk[curbs_mapped_on_sidewalk['OBJECTID'].isin(streets_w_1_curb['OBJECTID'])]
#curbs_1
#11578

In [100]:
problem_curbs = curbs_mapped_on_sidewalk[curbs_mapped_on_sidewalk['OBJECTID'].isin(streets_w_more_than_2_curbs['OBJECTID'])]
problem_curbs = problem_curbs[['attribute_id','label_type','OBJECTID','geometry', 'avg_label_date']]
problem_curbs

,attribute_id,label_type,OBJECTID,geometry,avg_label_date
1,70839937,CurbRamp,3812,POINT (1265474.804 212479.929),2019-04-22 00:58:33.914
2,70839941,CurbRamp,3812,POINT (1265333.281 212658.083),2019-04-26 11:45:42.880
9,70839953,CurbRamp,3812,POINT (1265457.555 212464.960),2019-04-17 04:35:25.223
15,70839965,CurbRamp,28845,POINT (1266780.059 212090.960),2019-04-26 21:52:27.473
16,70839966,CurbRamp,28845,POINT (1266420.221 212088.301),2019-04-26 21:51:37.253
...,...,...,...,...,...
44982,77219627,CurbRamp,11853,POINT (1283626.353 261726.837),2023-05-14 02:11:24.545
45005,77219655,CurbRamp,11853,POINT (1283609.241 261716.025),2023-05-14 02:11:30.063
44979,77219624,CurbRamp,777,POINT (1282587.613 261736.690),2022-03-30 14:38:54.352
44995,77219642,CurbRamp,777,POINT (1282580.337 261749.353),2019-12-22 20:11:08.510


In [101]:
problem_curbs_keep = problem_curbs .sort_values('avg_label_date').groupby('OBJECTID').tail(2).sort_values(by=['OBJECTID'])
problem_curbs_keep

,attribute_id,label_type,OBJECTID,geometry,avg_label_date
11851,77056063,CurbRamp,2,POINT (1273340.969 224267.286),2021-01-21 12:20:21.154
11954,77056170,CurbRamp,2,POINT (1273213.392 224484.081),2021-01-21 11:38:13.364
14748,77072180,NoCurbRamp,82,POINT (1260850.774 197124.795),2022-02-05 14:20:45.704
14746,77072178,NoCurbRamp,82,POINT (1261142.974 197124.529),2022-01-02 04:43:45.954
14768,77072200,NoCurbRamp,102,POINT (1259185.678 197195.698),2019-07-28 06:04:28.397
...,...,...,...,...,...
42458,77208061,CurbRamp,46217,POINT (1279031.463 200008.948),2023-04-02 06:03:59.776
1974,77017773,NoCurbRamp,46220,POINT (1280665.649 214833.135),2021-01-09 05:16:58.769
2002,77017804,NoCurbRamp,46220,POINT (1280678.616 214821.755),2021-01-09 05:17:01.844
654,77012425,CurbRamp,46243,POINT (1269592.516 226170.288),2021-04-04 15:06:39.968


In [102]:
problem_curbs_left = problem_curbs[~problem_curbs['attribute_id'].isin(problem_curbs_keep['attribute_id'])]
problem_curbs_left

,attribute_id,label_type,OBJECTID,geometry,avg_label_date
9,70839953,CurbRamp,3812,POINT (1265457.555 212464.960),2019-04-17 04:35:25.223
16,70839966,CurbRamp,28845,POINT (1266420.221 212088.301),2019-04-26 21:51:37.253
20,70839973,CurbRamp,28845,POINT (1266397.571 212085.963),2019-04-26 21:51:34.665
34,77011507,CurbRamp,17957,POINT (1270170.158 221816.800),2020-04-14 23:53:44.985
72,77011549,CurbRamp,17345,POINT (1269754.669 222232.687),2020-07-26 21:24:32.706
...,...,...,...,...,...
44892,77219370,NoCurbRamp,20494,POINT (1267558.189 254505.615),2019-10-29 01:35:11.094
44907,77219386,NoCurbRamp,19750,POINT (1267629.995 257390.676),2021-03-29 18:21:19.034
44959,77219602,CurbRamp,7369,POINT (1282909.217 259764.115),2019-11-30 21:17:57.145
44973,77219618,CurbRamp,11853,POINT (1283860.750 261803.143),2023-05-01 03:27:54.685


In [105]:
streets_w_1_curb_geom = sidewalk[sidewalk['OBJECTID'].isin(streets_w_1_curb['OBJECTID'])]

In [106]:
streets_w_1_curb_geom

,OBJECTID,COMPKEY,COMPTYPE,SEGKEY,UNITID,UNITTYPE,UNITDESC,ADDBY,ADDDTTM,ASBLT,CONDITION,CONDITION_,CURBTYPE,CURRENT_ST,CURRENT__1,FILLERTYPE,FILLERWID,INSTALL_DA,SW_WIDTH,MAINTAINED,MATL,MODBY,MODDTTM,OWNERSHIP,SIDE,SURFTYPE,BUILDERCD,MAINTBYRDW,SWINCOMPLE,MULTIPLESU,GSITYPECD,DATE_MVW_L,PRIMARYDIS,SECONDARYD,OVERRIDEYN,OVERRIDECO,SRTS_SIDEW,PRIMARYCRO,MINIMUMVAR,LAST_VERIF,COLOR,OWNERSHIP_,NATURE_OF_,MAINT_FINA,SW_CATEGOR,VARIABLEWI,MAINTENANC,SHAPE_Leng,geometry
2,3,328244,97,9308,SDW-43317,SDW,"BOREN AVE BETWEEN TERRACE ST AND JEFFERSON ST,...",SW DATA LOAD,2007-09-26,None,GOOD,2017-06-30,410A,INSVC,2010-07-31,LSCP,30,None,72,None,None,SIDEWALK COLLECTOR,2018-02-07,None,NE,PCC,None,N,N,N,None,2022-06-17,DISTRICT3,None,N,None,3.0,2.4,0,2017-06-30,None,None,None,None,BLOCK,N,SSRP_C4_SWShim,259.229392,"LINESTRING (1273407.188 224281.945, 1273275.19..."
4,5,328246,97,9307,SDW-30393,SDW,"BOREN AVE BETWEEN BROADWAY AND TERRACE ST, NE ...",SW DATA LOAD,2007-09-26,None,EXCELLENT,2017-06-30,410C,INSVC,2010-07-31,LSCP,18,None,72,None,None,SR1146066,2021-08-27,PARK,NE,PCC,None,N,N,N,None,2022-06-17,DISTRICT3,None,N,None,3.0,1.5,0,2017-06-30,None,2021-08-27,None,None,BLOCK,N,SSRP_C4_SWShim,121.582237,"LINESTRING (1273481.272 224156.709, 1273419.37..."
7,8,327860,97,5477,SDW-44786,SDW,35TH AVE SW BETWEEN SW ROSE ST AND SW THISTLE ...,SW DATA LOAD,2007-09-26,None,FAIR,2017-06-20,410C,INSVC,2007-09-26,LSCP,114,None,72,None,None,SIDEWALK COLLECTOR,2018-02-07,None,E,PCC,None,N,N,N,None,2022-06-17,DISTRICT1,None,N,None,3.0,1.7,0,2017-06-20,None,None,None,None,BLOCK,N,SSRP_SSW1_SWShim,217.377138,"LINESTRING (1259199.151 196765.686, 1259194.88..."
8,9,327861,97,9984,SDW-31572,SDW,DELRIDGE WAY SW BETWEEN SW KENYON ST AND SW TH...,SW DATA LOAD,2007-09-26,None,FAIR,2017-06-05,410C,INSVC,2007-09-26,LSCP,40,None,72,None,None,SIDEWALK COLLECTOR,2018-02-07,None,E,PCC,None,N,N,N,None,2022-06-17,DISTRICT1,None,N,None,3.0,2.2,0,2017-06-05,None,None,None,None,BLOCK,N,SSRP_SSW3_SWShim,1263.635930,"LINESTRING (1263210.980 197730.489, 1263181.68..."
13,14,327866,97,9982,SDW-31694,SDW,DELRIDGE WAY SW BETWEEN 21ST N AVE SW AND SW H...,SW DATA LOAD,2007-09-26,None,GOOD,2017-06-02,410C,INSVC,2007-09-26,LSCP,66,None,60,None,None,SR1146066,2021-08-27,PARK,W,PCC,None,N,N,N,None,2022-06-17,DISTRICT1,None,N,None,3.0,2.3,0,2017-06-02,None,2021-08-27,None,None,BLOCK,N,SSRP_SSW1_SWShim,124.602918,"LINESTRING (1263147.653 198557.487, 1263167.15..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46170,46232,322791,97,19813,SDW-42478,SDW,S DAWSON ST BETWEEN 52ND AVE S AND SEWARD PARK...,SW DATA LOAD,2007-09-26,None,FAIR,2017-06-21,410C,INSVC,2007-09-26,LSCP,18,None,72,None,None,SIDEWALK COLLECTOR,2018-02-07,None,S,PCC,None,N,N,N,None,2022-06-17,DISTRICT2,None,N,None,3.0,5.2,0,2017-06-21,None,None,None,None,BLOCK,N,SSRP_C3_SWShim,513.140972,"LINESTRING (1286165.374 205665.549, 1286678.40..."
46175,46237,324151,97,3960,SDW-28508,SDW,26TH AVE SW BETWEEN SW CAMBRIDGE ST AND SW ROX...,SW DATA LOAD,2007-09-26,None,FAIR,2017-06-22,410C,INSVC,2010-07-31,PCC,30,None,60,None,None,SIDEWALK COLLECTOR,2018-02-07,None,W,PCC,None,N,N,N,None,2022-06-17,DISTRICT1,None,N,None,3.0,0.2,0,2017-06-22,None,None,None,None,BLOCK,N,SSRP_SSW1_SWShim,606.490035,"LINESTRING (1261717.882 193126.892, 1261703.74..."
46178,46240,325818,97,11868,SDW-42084,SDW,"MINOR AVE BETWEEN JAMES ST AND CHERRY ST, SW SIDE",SW DATA LOAD,2007-09-26,None,FAIR,2017-06-30,MONO,INSVC,2010-07-31,NONE,0,None,90,None,None,SIDEWALK COLLECTOR,2018-02-07,None,SW,PCC,None,N,N,N,None,2022-06-17,DISTRICT3,None,N,None,1.0,1.6,0,2017-06-30,None,None,None,None,BLOCK,N,SSRP_C4_SWShim,279.993952,"LINESTRING (1273318.702 224949.203, 1273176.19..."
46185,46247,330578,97,11821,SDW-36263,SDW,MERIDIAN AVE N BETWEEN N 90TH N ST AND N 92ND ...,SW DATA LOAD,2007-09-26,None,None,None,NONE,INSVC,

In [107]:
#spatial join the streets with fewer than 2 curbs with the problem_curbs_left sjoin_nearest
second_attempt = problem_curbs_left.sjoin_nearest(streets_w_1_curb_geom , distance_col='distance', max_distance = 30)

In [108]:
second_attempt 

,attribute_id,label_type,OBJECTID_left,geometry,avg_label_date,index_right,OBJECTID_right,COMPKEY,COMPTYPE,SEGKEY,UNITID,UNITTYPE,UNITDESC,ADDBY,ADDDTTM,ASBLT,CONDITION,CONDITION_,CURBTYPE,CURRENT_ST,CURRENT__1,FILLERTYPE,FILLERWID,INSTALL_DA,SW_WIDTH,MAINTAINED,MATL,MODBY,MODDTTM,OWNERSHIP,SIDE,SURFTYPE,BUILDERCD,MAINTBYRDW,SWINCOMPLE,MULTIPLESU,GSITYPECD,DATE_MVW_L,PRIMARYDIS,SECONDARYD,OVERRIDEYN,OVERRIDECO,SRTS_SIDEW,PRIMARYCRO,MINIMUMVAR,LAST_VERIF,COLOR,OWNERSHIP_,NATURE_OF_,MAINT_FINA,SW_CATEGOR,VARIABLEWI,MAINTENANC,SHAPE_Leng,distance
9,70839953,CurbRamp,3812,POINT (1265457.555 212464.960),2019-04-17 04:35:25.223,5294,5295,289203,97,1815,SDW-613,SDW,13TH AVE SW BETWEEN 11TH AV SW ACCESS RD AND K...,SW DATA LOAD,2007-09-26,None,EXCELLENT,2017-07-06,410A,INSVC,2007-09-26,NONE,0,None,84,None,None,SIDEWALK COLLECTOR,2018-02-07,None,W,PCC,None,N,N,N,None,2022-06-17,DISTRICT1,None,N,None,1.0,0.8,0,2017-07-06,None,None,None,None,BLOCK,N,SSRP_SSW3_SWShim,184.913578,29.981843
20,70839973,CurbRamp,28845,POINT (1266397.571 212085.963),2019-04-26 21:51:34.665,45617,45626,318264,97,1509,SDW-23270,SDW,11TH CR AVE SW BETWEEN SW SPOKANE ST AND SW KL...,SW DATA LOAD,2007-09-26,None,None,None,410C,INSVC,None,None,0,None,0,None,None,FEGLEYC,2017-07-26,None,E,UIMPRV,None,N,N,N,None,2022-06-17,DISTRICT1,None,N,None,4.0,0.0,0,None,None,None,None,None,IMPCRNR,N,None,71.980962,12.090258
61,77011538,CurbRamp,10711,POINT (1270198.319 223067.419),2021-03-23 22:29:14.476,41894,41901,327086,97,1185,SDW-32769,SDW,1ST AVE S BETWEEN YESLER WAY AND S WASHINGTON ...,SW DATA LOAD,2007-09-26,None,POOR,2017-06-22,G,INSVC,2010-07-31,PCC,42,None,142,None,None,SIDEWALK COLLECTOR,2018-02-07,None,E,PCC,None,N,N,Y,None,2022-06-17,DISTRICT7,None,N,None,3.0,6.4,0,2017-06-22,None,None,None,None,BLOCK,N,SSRP_C5_SWShim,269.204232,4.331484
65,77011542,CurbRamp,2181,POINT (1270446.769 222195.525),2020-05-19 20:41:36.523,4050,4051,666212,97,20482,SDW-46292,SDW,S JACKSON ST BETWEEN 1ST AVE S AND OCCIDENTAL ...,FEGLEYC,2016-12-05,774-775,EXCELLENT,2017-06-23,None,INSVC,2015-06-01,NONE,0,2014-06-27,144,None,None,SIDEWALK COLLECTOR,2018-02-07,None,C,PCC,CIP-SDOT,N,N,N,None,2022-06-17,DISTRICT2,DISTRICT7,N,None,3.0,1.0,102,2017-06-23,None,None,None,None,TRPLATFM,Y,SSRP_C5_SWShim,177.516138,20.202113
99,77011579,CurbRamp,2177,POINT (1271083.034 222191.484),2020-01-18 05:33:30.636,44825,44832,657327,97,4516,SDW-45906,SDW,"3RD AVE S BETWEEN S MAIN ST AND S JACKSON ST, ...",WANGD1,2016-08-01,None,FAIR,2017-06-22,410C,INSVC,2016-08-01,TR/PCC,175,None,95,None,None,SIDEWALK COLLECTOR,2018-02-07,None,W,PCC,None,N,N,N,None,2022-06-17,DISTRICT7,None,N,None,3.0,3.8,0,2017-06-22,None,None,None,None,BLOCK,N,SSRP_C5_SWShim,168.759778,16.904958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44585,77215948,CurbRamp,4133,POINT (1265626.978 260820.489),2022-04-24 01:03:55.133,25646,25647,310766,97,10934,SDW-39696,SDW,HOLMAN RD N BETWEEN N 104TH ST AND GREENWOOD A...,SW DATA LOAD,2007-09-26,None,FAIR,2017-08-08,410A,INSVC,2007-09-26,LSCP,60,None,60,None,None,SIDEWALK COLLECTOR,2018-02-07,None,SE,PCC,None,N,N,N,None,2022-06-17,DISTRICT6,None,N,None,3.0,2.4,0,2017-08-08,None,None,None,None,BLOCK,N,SSRP_NW5_SWShim,311.987998,20.294777
44892,77219370,NoCurbRamp,20494,POINT (1267558.189 254505.615),2019-10-29 01:35:11.094,20220,20221,305297,97,15915,SDW-11810,SDW,N 81ST ST BETWEEN FREMONT AVE N AND LINDEN AVE...,SW DATA LOAD,2007-09-26,None,FAIR,2017-07-26,410C,INSVC,2007-09-26,LSCP,108,None,72,None,None,SIDEWALK COLLECTOR,2018-02-07,None,N,PCC,None,N,N,N,None,2022-06-17,DISTRICT6,None,N,None,1.0,1.3,0,2017-07-26,None,None,None,None,BLOCK,N,SSRP_NW5_SWShim,636.929686,20.631158
44907,77219386,NoCurbRamp,19750,POINT (1267629.995 257390.676),2021-03-29 18:21:19.034,20427,20428,305342,97,16029,SDW-11965,SDW,N 92ND ST BETWEEN L

In [159]:
second_attempt = second_attempt [['attribute_id','label_type','OBJECTID_right','geometry', 'avg_label_date']]
second_attempt

,attribute_id,label_type,OBJECTID_right,geometry,avg_label_date
9,70839953,CurbRamp,5295,POINT (1265457.555 212464.960),2019-04-17 04:35:25.223
20,70839973,CurbRamp,45626,POINT (1266397.571 212085.963),2019-04-26 21:51:34.665
61,77011538,CurbRamp,41901,POINT (1270198.319 223067.419),2021-03-23 22:29:14.476
65,77011542,CurbRamp,4051,POINT (1270446.769 222195.525),2020-05-19 20:41:36.523
99,77011579,CurbRamp,44832,POINT (1271083.034 222191.484),2020-01-18 05:33:30.636
...,...,...,...,...,...
44585,77215948,CurbRamp,25647,POINT (1265626.978 260820.489),2022-04-24 01:03:55.133
44892,77219370,NoCurbRamp,20221,POINT (1267558.189 254505.615),2019-10-29 01:35:11.094
44907,77219386,NoCurbRamp,20428,POINT (1267629.995 257390.676),2021-03-29 18:21:19.034
44959,77219602,CurbRamp,7975,POINT (1282909.217 259764.115),2019-11-30 21:17:57.145


In [161]:
second_attempt_keep = second_attempt .sort_values('avg_label_date').groupby('OBJECTID_right').tail(1).sort_values(by=['OBJECTID_right'])
second_attempt_keep

,attribute_id,label_type,OBJECTID_right,geometry,avg_label_date
14790,77072222,NoCurbRamp,80,POINT (1260850.913 197131.751),2020-11-20 06:04:44.587
9458,77043723,NoCurbRamp,225,POINT (1271219.907 204918.825),2022-04-20 19:25:07.452
23016,77111876,NoCurbRamp,372,POINT (1254575.144 208644.069),2019-05-15 06:21:47.305
9278,77041931,CurbRamp,402,POINT (1269577.124 205518.648),2021-12-10 19:32:26.067
25102,77125091,CurbRamp,716,POINT (1276126.525 249314.801),2019-04-28 18:04:10.304
...,...,...,...,...,...
15820,77077052,CurbRamp,45963,POINT (1274843.335 227109.421),2021-01-28 00:08:19.917
42866,77211872,CurbRamp,45991,POINT (1270916.143 260155.702),2022-04-15 05:34:12.177
3234,77024133,CurbRamp,46191,POINT (1272456.866 236914.369),2020-06-21 02:08:11.725
21563,77102460,CurbRamp,46202,POINT (1273293.522 230084.242),2019-11-28 04:45:32.451


In [109]:
#count the number of curbs per street
second_attempt_count_per_street= second_attempt.groupby(['OBJECTID_right']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
second_attempt_count_per_street

,OBJECTID_right,counts
1995,45140,5
1343,25769,4
886,19119,4
78,4894,4
476,11310,4
...,...,...
754,16913,1
753,16902,1
752,16900,1
750,16890,1


In [157]:
# #select second_attempt_count_per_street where counts is 1
# second_attempt_count_per_street_1 = second_attempt_count_per_street[second_attempt_count_per_street['counts'] == 1]
# second_attempt_count_per_street_1

1603

In [113]:
# #select these in second_attempt
# second_attempt_1 = second_attempt[second_attempt['OBJECTID_right'].isin(second_attempt_count_per_street_1['OBJECTID_right'])]
# second_attempt_1 = second_attempt_1[['attribute_id','OBJECTID_right']]
# second_attempt_1 = second_attempt_1.rename(columns={'OBJECTID_right': 'OBJECTID'})

In [164]:
second_attempt_keep= second_attempt_keep.rename(columns={'OBJECTID_right': 'OBJECTID'})
second_attempt_keep

,attribute_id,label_type,OBJECTID,geometry,avg_label_date
14790,77072222,NoCurbRamp,80,POINT (1260850.913 197131.751),2020-11-20 06:04:44.587
9458,77043723,NoCurbRamp,225,POINT (1271219.907 204918.825),2022-04-20 19:25:07.452
23016,77111876,NoCurbRamp,372,POINT (1254575.144 208644.069),2019-05-15 06:21:47.305
9278,77041931,CurbRamp,402,POINT (1269577.124 205518.648),2021-12-10 19:32:26.067
25102,77125091,CurbRamp,716,POINT (1276126.525 249314.801),2019-04-28 18:04:10.304
...,...,...,...,...,...
15820,77077052,CurbRamp,45963,POINT (1274843.335 227109.421),2021-01-28 00:08:19.917
42866,77211872,CurbRamp,45991,POINT (1270916.143 260155.702),2022-04-15 05:34:12.177
3234,77024133,CurbRamp,46191,POINT (1272456.866 236914.369),2020-06-21 02:08:11.725
21563,77102460,CurbRamp,46202,POINT (1273293.522 230084.242),2019-11-28 04:45:32.451


In [170]:
problem_curbs_keep = problem_curbs_keep[['attribute_id','OBJECTID']]
curbs_2 = curbs_2[['attribute_id','OBJECTID']]
curbs_1 = curbs_1[['attribute_id','OBJECTID']]
second_attempt_keep = second_attempt_keep[['attribute_id','OBJECTID']]

In [171]:
#concact second_attempt_1, problem_curbs_keep, curbs_2, curbs_1
curbs_to_sidewalk = pd.concat([second_attempt_keep, problem_curbs_keep, curbs_2, curbs_1])
curbs_to_sidewalk

,attribute_id,OBJECTID
14790,77072222,80
9458,77043723,225
23016,77111876,372
9278,77041931,402
25102,77125091,716
...,...,...
45038,77221359,12158
45042,77221365,12118
45044,77221368,7376
45045,77221370,13471


In [167]:
#count the number of curbs per street
new = curbs_to_sidewalk.groupby(['OBJECTID']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
new['counts'].value_counts()

2    15456
1     9562
Name: counts, dtype: int64

In [172]:
curbs_to_sidewalk.to_csv('curbs_to_sidewalk.csv', index=False)

In [132]:
#selecte the attribute_id in curbs
curbs_filtered = curbs_mapped_on_sidewalk[curbs_mapped_on_sidewalk['attribute_id'].isin(curbs_to_sidewalk['attribute_id'])]
sidewalk_filtered = sidewalk[sidewalk['OBJECTID'].isin(curbs_to_sidewalk['OBJECTID'])]

In [133]:
#change avg_image_capture_date and avg_label_date to isoformat .apply(lambda x: x.isoformat())
curbs_filtered['avg_image_capture_date'] = curbs_filtered['avg_image_capture_date'].apply(lambda x: x.isoformat())
curbs_filtered['avg_label_date'] = curbs_filtered['avg_label_date'].apply(lambda x: x.isoformat())

/Users/chuli/opt/anaconda3/envs/project-sidewalk/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [142]:
#change curbs_filtered and sidewalk_filtered to epsg 4326
curbs_filtered = curbs_filtered.to_crs(epsg=4326)
sidewalk_filtered = sidewalk_filtered.to_crs(epsg=4326)

In [143]:
q1_curbs = curbs_filtered.cx[-122.323 : -122.418, 47.734: 47.649]
# q2 = curbs.cx[-122.239 : -122.323, 47.734: 47.649]
# q3 = curbs.cx[-122.239 : -122.323, 47.651: 47.495]
# q4 = curbs.cx[-122.323 : -122.418, 47.651: 47.495]


In [144]:
q1_sidewalk = sidewalk_filtered.cx[-122.323 : -122.418, 47.734: 47.649]

In [145]:
base = q1_sidewalk.explore(color='black', tiles='cartodbpositron',tooltip=['OBJECTID'])

In [147]:
q1 = q1_curbs.explore(tiles='cartodbpositron', column ='label_type', tooltip=['OBJECTID'], m=base)
q1.save('q1.html')